# Example of generating QAs for a Paul Graham Essay
**Source:** http://www.paulgraham.com/makersschedule.html

**Description:** A famous essay by Paul Graham about the difference between the schedules of managers and makers.

### Before running the code

You will need to have the following packages installed:
```
pip install langchain pandas pypdf
```

Also, make sure you have a .env file with your OpenAI API key in the root directory of this project.
```
OPENAI_API_KEY=YOUR_API_KEY
```

### Load Packages

In [3]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [4]:
import os
import pandas as pd
from dotenv import load_dotenv
from uniflow.flow.client import TransformClient
from uniflow.flow.config import TransformOpenAIConfig
from uniflow.op.model.model_config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.op.prompt import Context, PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

### Prepare the input data

In [5]:
pdf_file = "makers_schedule_managers_schedule.pdf"

Set current directory and input data directory.

In [6]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

In [7]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()

In [8]:
guided_prompt = PromptTemplate(
    instruction="Generate one question and its corresponding answer based on the context. Following the format of the examples below to include the same context, question, and answer in the response.",
    few_shot_prompt=[
        Context(
            context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.",
            question="Who published A Mathematical Theory of Communication in 1948?",
            answer="Claude E. Shannon."
        ),
    ]
)

data = [ Context(context=p) for p in pages[0].page_content.split("\n\n") if len(p) > 200]


In [9]:
data

[Context(context='11/6/23, 11:40 AM Maker\'s Schedule, Manager\'s Schedule\nhttps://www.paulgraham.com/makersschedule.html 1/3\n"...the mere consciousness of an engagement will sometimes\nworry a whole da y."\n– Charles Dick ens\nJuly 2009\nOne reason progr ammers dislik e meetings so much is that they\'re\non a different t ype of schedule from other people. Meetings cost\nthem more.\nThere are two t ypes of schedule, which I\'ll call the manager\'s\nschedule and the mak er\'s schedule. The manager\'s schedule is for\nbosses. It\'s embodied in the tr aditional appointment book, with\neach da y cut into one hour interv als. Y ou can block off sev eral\nhours for a single task if y ou need to , but b y default y ou change\nwhat y ou\'re doing ev ery hour .\nWhen y ou use time that w ay, it\'s merely a pr actical problem to\nmeet with someone. Find an open slot in y our schedule, book\nthem, and y ou\'re done.\nMost powerful people are on the manager\'s schedule. It\'s the\nschedule of co

In [10]:
config = TransformOpenAIConfig(
    model_config=OpenAIModelConfig(response_format={"type": "json_object"}),
)
client = TransformClient(config)

In [11]:
output = client.run(data)

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
output


[{'output': [{'response': [{'context': '11/6/23, 11:40 AM Maker\'s Schedule, Manager\'s Schedule\nhttps://www.paulgraham.com/makersschedule.html 1/3\n"...the mere consciousness of an engagement will sometimes\nworry a whole da y."\nCharles Dick ens\nJuly 2009\nOne reason progr ammers dislik e meetings so much is that they\'re\non a different t ype of schedule from other people. Meetings cost\nthem more.\nThere are two t ypes of schedule, which I\'ll call the manager\'s\nschedule and the mak er\'s schedule. The manager\'s schedule is for\nbosses. It\'s embodied in the tr aditional appointment book, with\neach da y cut into one hour interv als. Y ou can block off sev eral\nhours for a single task if y ou need to , but b y default y ou change\nwhat y ou\'re doing ev ery hour .\nWhen y ou use time that w ay, it\'s merely a pr actical problem to\nmeet with someone. Find an open slot in y our schedule, book\nthem, and y ou\'re done.\nMost powerful people are on the manager\'s schedule. It\'s

In [13]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

for item in output:
    for i in item['output']:
        for response in i['response']:
            contexts.append(response['context'])
            questions.append(response['question'])
            answers.append(response['answer'])

df = pd.DataFrame({
    'context': contexts,
    'question': questions,
    'answer': answers
})

# Set display options
pd.set_option('display.max_colwidth', None)  # or use a specific width like 50
pd.set_option('display.width', 1000)

styled_df = df.head().style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])
styled_df

,context,question,answer
0,"11/6/23, 11:40 AM Maker's Schedule, Manager's Schedule https://www.paulgraham.com/makersschedule.html 1/3 ""...the mere consciousness of an engagement will sometimes worry a whole da y."" Charles Dick ens July 2009 One reason progr ammers dislik e meetings so much is that they're on a different t ype of schedule from other people. Meetings cost them more. There are two t ypes of schedule, which I'll call the manager's schedule and the mak er's schedule. The manager's schedule is for bosses. It's embodied in the tr aditional appointment book, with each da y cut into one hour interv als. Y ou can block off sev eral hours for a single task if y ou need to , but b y default y ou change what y ou're doing ev ery hour . When y ou use time that w ay, it's merely a pr actical problem to meet with someone. Find an open slot in y our schedule, book them, and y ou're done. Most powerful people are on the manager's schedule. It's the schedule of command. But there's another w ay of using time that's common among people who mak e things, lik e progr ammers and writers. They gener ally prefer to use time in units of half a day at least. Y ou can't write or progr am well in units of an hour . That's barely enough time to get started. When y ou're oper ating on the mak er's schedule, meetings are a disaster . A single meeting can blow a whole afternoon, b y breaking it into two pieces each too small to do an ything hard in. Plus y ou ha ve to remember to go to the meeting. That's no problem for someone on the manager's schedule. There's alw ays something coming on the next hour; the only question is what. But when someone on the mak er's schedule has a meeting, they have to think about it. For someone on the mak er's schedule, ha ving a meeting is lik e throwing an ex ception. It doesn't merely cause y ou to switch from one task to another; it changes the mode in which y ou work. I find one meeting can sometimes affect a whole da y. A meeting commonly blows at least half a da y, by breaking up a morning or afternoon. But in addition there's sometimes a cascading effect. If I know the afternoon is going to be brok en up , I'm slightly less likely to start something ambitious in the morning. I know this may sound o versensitiv e, but if y ou're a mak er, think of y our own case. Don't y our spirits rise at the thought of ha ving an entire da y free to work, with no appointments at all? W ell, that means y our spirits are correspondingly depressed when y ou don't. And ambitious projects are b y definition close to the limits of y our",What type of schedule do programmers and writers prefer to use?,They generally prefer to use time in units of half a day at least.
